In [1]:
!pip install nlp 
!pip install tokenizers
!pip install transformers

     |████████████████████████████████| 1.7MB 2.8MB/s 
     |████████████████████████████████| 245kB 18.5MB/s 
     |████████████████████████████████| 17.2MB 239kB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1
     |████████████████████████████████| 3.0MB 2.8MB/s 
     |████████████████████████████████| 778kB 2.8MB/s 
     |████████████████████████████████| 890kB 14.3MB/s 
     |████████████████████████████████| 3.0MB 17.7MB/s 
     |████████████████████████████████| 1.1MB 35.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=6939942175857abf3badcd25cd84bca5ceec305057c0b19ee147d1d5afbbe96c
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
  Found existing installation: tokenizers 0.8.1
    Uninstalling tokenizers-0.8.1:
      Successfully uninstalled tokenizers-0.8.1


In [2]:
import os
import torch

import nlp
import tokenizers
import transformers

In [4]:
mnli = nlp.load_dataset(path='glue', name='mnli', split='train[:50%]')

In [9]:
from transformers import XLMRobertaTokenizer
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-large')

In [10]:
def convert_to_features(batch):
    input_pairs = list(zip(batch['premise'], batch['hypothesis']))
    encodings = tokenizer.batch_encode_plus(input_pairs, 
                                            add_special_tokens=True, 
                                            padding=True, 
                                            max_length=96, 
                                            truncation=True, 
                                            return_attention_mask=True, 
                                            return_token_type_ids=True)
    return encodings

### MNLI

In [253]:
mnli_encoded_dataset = mnli.map(convert_to_features, batched=True, remove_columns=['idx', 'premise', 'hypothesis'])
mnli_encoded_dataset.set_format("torch", columns=['attention_mask', 'input_ids', 'token_type_ids', 'label'])

In [254]:
print(mnli_encoded_dataset.num_rows)
print(mnli_encoded_dataset.num_columns)
print(mnli_encoded_dataset.column_names)

196351
4
['label', 'input_ids', 'token_type_ids', 'attention_mask']


### XNLI

In [255]:
xnli = nlp.load_dataset(path='xnli')
xnli = nlp.concatenate_datasets([xnli['test'], xnli['validation']])

In [256]:
def preprocess_xnli(example):
    premise_output = []
    hypothesis_output = []
    label_output = []
    for prem, hyp, lab in zip(example['premise'],  example['hypothesis'], example["label"]):
        label = lab
        langs = hyp['language']
        translations = hyp['translation']
        hypothesis = {k: v for k, v in zip(langs, translations)}
        for lang in prem:
            if lang in hypothesis:
                premise_output += [prem[lang]]
                hypothesis_output += [hypothesis[lang]]
                label_output += [label]
    return {'premise':premise_output, 'hypothesis':hypothesis_output, 'label':label_output}

In [257]:
xnli_processed = xnli.map(preprocess_xnli, batched=True).shuffle(seed=2020)
xnli_encoded = xnli_processed.map(convert_to_features, batched=True, remove_columns=['premise', 'hypothesis'])
xnli_encoded.set_format("torch", columns=['attention_mask', 'input_ids', 'token_type_ids', 'label']) 

In [258]:
print(xnli_processed.num_rows)
print(xnli_processed.num_columns)
print(xnli_processed.column_names)

print(xnli_encoded.num_rows)
print(xnli_encoded.num_columns)
print(xnli_encoded.column_names)

112500
3
['hypothesis', 'label', 'premise']
112500
4
['label', 'input_ids', 'token_type_ids', 'attention_mask']


## The Stanford Natural Language Inference Corpus (SNLI)

In [259]:
snli = nlp.load_dataset(path='snli', split='train[:20%]')

In [260]:
print(len(snli.filter(lambda x: x['label']==0)))
print(len(snli.filter(lambda x: x['label']==1)))
print(len(snli.filter(lambda x: x['label']==2)))

36726
36524
36658


In [262]:
snli_encoded = snli.map(convert_to_features, batched=True, remove_columns=['premise', 'hypothesis'])
snli_encoded.set_format("torch", columns=['attention_mask', 'input_ids', 'token_type_ids', 'label']) 

### Check all three datasets

In [263]:
mnli_encoded_dataset.num_rows + snli_encoded.num_rows + xnli_encoded.num_rows

418881

In [264]:
print(mnli_encoded_dataset.column_names)
print(snli_encoded.column_names)
print(xnli_encoded.column_names)

['label', 'input_ids', 'token_type_ids', 'attention_mask']
['label', 'input_ids', 'token_type_ids', 'attention_mask']
['label', 'input_ids', 'token_type_ids', 'attention_mask']


In [266]:
dataset = nlp.concatenate_datasets([mnli_encoded_dataset, snli_encoded, xnli_encoded])

In [268]:
dataset = dataset.shuffle(seed=2020)